# Visualizing Neo4j Graph Data Science (GDS) Graphs

In [ ]:
# make sure to install the graphdatascience package
%pip install graphdatascience

## Setup GDS graph

In [ ]:
import os

from graphdatascience import GraphDataScience

# Get Neo4j DB URI, credentials and name from environment if applicable
NEO4J_URI = os.environ.get("NEO4J_URI", "bolt://localhost:7687")
NEO4J_AUTH = ("neo4j", None)
NEO4J_DB = os.environ.get("NEO4J_DB", "neo4j")
if os.environ.get("NEO4J_USER") and os.environ.get("NEO4J_PASSWORD"):
    NEO4J_AUTH = (
        os.environ.get("NEO4J_USER"),
        os.environ.get("NEO4J_PASSWORD"),
    )
gds = GraphDataScience(NEO4J_URI, auth=NEO4J_AUTH, database=NEO4J_DB)

In [ ]:
G = gds.graph.load_cora()

In [ ]:
# Run some algorithms to use later for visualization
gds.pageRank.mutate(G, mutateProperty="pagerank")
gds.louvain.mutate(G, mutateProperty="componentId")

For visualization we recommend limiting the number of nodes to 10k.
A good way to create a smaller subgraph is by using the [sampling](https://neo4j.com/docs/graph-data-science/current/management-ops/graph-creation/sampling/) or [filtering](https://neo4j.com/docs/graph-data-science/current/management-ops/graph-creation/graph-filter/) the GDS graph.

In [ ]:
G_sample, _ = gds.graph.sample.rwr("sampled", G)

In [ ]:
G_sample.node_properties()


## Visualization

In [ ]:
from neo4j_viz.gds import from_gds

# TODO inform about automatic size scaling
VG = from_gds(gds, G_sample, size_property="pagerank", additional_node_properties=["componentId", "subject"])

In [ ]:
VG.render()

### Adding captions

### Sizing

In [ ]:
VG.render()

### Coloring

multiple ways to color

* default colors
* provide custom color map (such as from palettable), such as if you exceed limit

In [ ]:
VG.color_nodes("subject")

In [ ]:
# map community to color
import colorsys
import random


def _random_themed_color(theme) -> str:
    label = 0.7 if theme == "dark" else 0.4
    return "#%02X%02X%02X" % tuple(map(lambda x: int(x * 255), colorsys.hls_to_rgb(random.random(), label, 1.0)))


color_dict = {component: _random_themed_color("dark") for _, component in node_df["componentId"].items()}
color_dict

In [ ]:
VG.color_nodes("component_id", color_dict)

In [ ]:
VG.render()

In [ ]:
from neo4j_viz import Layout

VG.render(layout=Layout.HIERARCHICAL, initial_zoom=0.1, pan_position=(2000, 0))

In [ ]:
VG.render(layout=Layout.GRID, initial_zoom=0.3, pan_position=(1000, 1000))

## Saving the visualization

In [ ]:
# Save the visualization to a file
with open("out/cora.html", "w") as f:
    f.write(VG.render().data)

## Cleanup

Lets cleanup the graphs we created in GDS.

In [ ]:
G.drop()
G_sample.drop()

In [ ]:
# TODO remove
gds.graph.drop("sampled")
gds.graph.drop("cora")